# 교차 검증 실습

In [14]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split,StratifiedKFold

In [ ]:
fish_df = pd.read_csv('./data/fish.csv')
fish_input = fish_df.drop('Species', axis=1)
fish_target = fish_df['Species']

### 생선 다중 분류 with cross_val_score

In [7]:
X_train,X_test, y_train,y_test = train_test_split(fish_input, fish_target, random_state=42, test_size=0.2)

In [12]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
# ~
scaler = StandardScaler()
X_scaled_train = scaler.fit_transform(X_train)
X_scaled_test = scaler.fit_transform(X_test)

lr = LogisticRegression(max_iter=1000, solver='newton-cg')
lr.fit(X_scaled_train,y_train)

# 교차검증 검수
cv = cross_val_score(lr, X_scaled_train,y_train,scoring='accuracy',cv=3)
print(f'cv : {cv}\n')
print(accuracy_score(y_test,y_pred))
# 예측 결과
y_pred = lr.predict(X_scaled_test)
print(f'y_pred : {y_pred}')

cv : [0.86046512 0.78571429 0.69047619]

0.875
y_pred : ['Perch' 'Smelt' 'Pike' 'Perch' 'Perch' 'Bream' 'Smelt' 'Perch' 'Perch'
 'Pike' 'Bream' 'Perch' 'Bream' 'Parkki' 'Bream' 'Bream' 'Perch' 'Perch'
 'Perch' 'Bream' 'Smelt' 'Bream' 'Bream' 'Bream' 'Bream' 'Perch' 'Perch'
 'Perch' 'Smelt' 'Smelt' 'Pike' 'Perch']


### 생선 다중 분류 with GridSearchCV

In [16]:
from sklearn.model_selection import GridSearchCV
# GridSearchCV 사용 -> 최적의 파라미터, 평가 점수, 모델 확인
parameter = {
    'max_iter' : [10,100,1000,1500],
    'solver' : ['liblinear', 'newton-cg', 'lbfgs']
}
lr = LogisticRegression()
stratified_kfold = StratifiedKFold(n_splits=5)
gscv = GridSearchCV(lr,param_grid=parameter,scoring='accuracy',cv=stratified_kfold)
gscv.fit(X_scaled_train, y_train)


# 예측 결과 평가

print('최적의 파라미터 : ', gscv.best_params_)
print('최적화된 모델 객체 : ', gscv.best_estimator_)
print('최적화된 점수: ', gscv.best_score_)
best_model = gscv.best_estimator_
best_model.fit(X_scaled_train, y_train)
print(best_model.score(X_scaled_test, y_test))

c:\Users\ljh10\anaconda3\envs\pystudy_env\Lib\site-packages\sklearn\model_selection\_split.py:776: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(
c:\Users\ljh10\anaconda3\envs\pystudy_env\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\ljh10\anaconda3\envs\pystudy_env\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale th

최적의 파라미터 :  {'max_iter': 10, 'solver': 'newton-cg'}
최적화된 모델 객체 :  LogisticRegression(max_iter=10, solver='newton-cg')
최적화된 점수:  0.7864615384615384
0.875


---

### HyperOpt

In [17]:
!pip install hyperopt

   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ---------------------------------------- 1.6/1.6 MB 42.6 MB/s eta 0:00:00


---

### Optuna